References:
https://www.youtube.com/watch?v=CbTU92pbDKw
https://arxiv.org/pdf/2203.12105

In [6]:
from miditok import REMI, TokenizerConfig
from miditoolkit import MidiFile
from pathlib import Path
import numpy as np

# --- Tokenizer config ---
config = TokenizerConfig(
    use_chords=True,
    use_rests=True,
    use_tempos=True,
    use_time_signatures=True,
    nb_tempos=32,
    tempo_range=(40, 250),
    chord_types='all'
)

tokenizer = REMI(config)

# --- Tokenize all MIDI files directly in memory ---
midi_folder = Path("/Users/macbook/Downloads/stock-prediction/")
all_token_ids = []

for midi_path in midi_folder.glob("*.mid"):
    try:
        midi = MidiFile(midi_path)
        tokens = tokenizer(midi)  # may return list or single

        if isinstance(tokens, list):
            for seq in tokens:
                all_token_ids.extend(seq.ids)
        else:
            all_token_ids.extend(tokens.ids)

    except Exception as e:
        print(f"Error with {midi_path.name}: {e}")



/var/folders/7z/4qfck68s789217tp5v__qhfw0000gn/T/ipykernel_2262/2753730279.py:7: UserWarning: Argument nb_tempos has been renamed num_tempos, you should consider to updateyour code with this new argument name.
  config = TokenizerConfig(
/var/folders/7z/4qfck68s789217tp5v__qhfw0000gn/T/ipykernel_2262/2753730279.py:26: UserWarning: You are using a depreciated `miditoolkit.MidiFile` object. MidiTokis now (>v3.0.0) using symusic.Score as MIDI backend. Your file willbe converted on the fly, however please consider using symusic.
  tokens = tokenizer(midi)  # may return list or single


In [7]:
seq_len = 64  # Sequence length
X, y = [], []

for i in range(len(all_token_ids) - seq_len):
    X.append(all_token_ids[i:i + seq_len])
    y.append(all_token_ids[i + seq_len])

X = np.array(X)
y = np.array(y)

print(f"X shape: {X.shape}, y shape: {y.shape}")

vocab_size = tokenizer.vocab_size

X shape: (5577, 64), y shape: (5577,)


In [10]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, shuffle=False
)

In [13]:
from keras.models import Sequential
from keras.optimizers import Adam
from keras import layers
from keras.callbacks import EarlyStopping
import tensorflow as tf


# --- Define model ---
# model = Sequential([
#     layers.Input(shape=(3, 2)),
#     layers.Bidirectional(layers.LSTM(128, return_sequences=True)),
#     layers.Bidirectional(layers.LSTM(64)),
#     layers.BatchNormalization(),
#     layers.Dense(128, activation='relu'),
#     layers.Dropout(0.3),
#     layers.Dense(64, activation='relu'),
#     layers.Dense(2)
# ])

model = Sequential([
  layers.Embedding(input_dim=vocab_size, output_dim=256, input_length=seq_len),
  layers.LSTM(512, return_sequences=True),
  layers.LSTM(512, return_sequences=True),
  layers.LSTM(512),
  layers.Dropout(0.3),
  layers.BatchNormalization(),
  layers.Dense(256),
  layers.Dropout(0.3),
  layers.Dense(128),
  layers.Dropout(0.3),
  layers.Activation(activation='relu'),
  layers.BatchNormalization(),
  layers.Dense(vocab_size, activation='softmax')
])

model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=Adam(learning_rate=0.001),
    metrics=['accuracy']
)

# --- Early stopping ---
# early_stop = EarlyStopping(
#     monitor='val_loss',
#     patience=10,
#     restore_best_weights=True
# )

log_dir = "logs/fit/0.0001-300epoch"
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

# --- Train ---
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=100,
    # callbacks=[early_stop]
    callbacks=[tensorboard_callback]
)


Epoch 1/100
140/140 ━━━━━━━━━━━━━━━━━━━━ 189s 1s/step - accuracy: 0.0149 - loss: 6.1081 - val_accuracy: 0.1971 - val_loss: 4.6852
Epoch 2/100
140/140 ━━━━━━━━━━━━━━━━━━━━ 162s 1s/step - accuracy: 0.2326 - loss: 4.0885 - val_accuracy: 0.3082 - val_loss: 3.1219
Epoch 3/100
140/140 ━━━━━━━━━━━━━━━━━━━━ 175s 1s/step - accuracy: 0.3698 - loss: 2.4846 - val_accuracy: 0.3575 - val_loss: 2.6654
Epoch 4/100
140/140 ━━━━━━━━━━━━━━━━━━━━ 169s 1s/step - accuracy: 0.3843 - loss: 2.2798 - val_accuracy: 0.3593 - val_loss: 2.5388
Epoch 5/100
140/140 ━━━━━━━━━━━━━━━━━━━━ 158s 1s/step - accuracy: 0.4029 - loss: 2.1172 - val_accuracy: 0.3522 - val_loss: 2.4471
Epoch 6/100
140/140 ━━━━━━━━━━━━━━━━━━━━ 157s 1s/step - accuracy: 0.4091 - loss: 2.0436 - val_accuracy: 0.3548 - val_loss: 2.4117
Epoch 7/100
140/140 ━━━━━━━━━━━━━━━━━━━━ 155s 1s/step - accuracy: 0.4231 - loss: 1.9314 - val_accuracy: 0.3943 - val_loss: 2.3537
Epoch 8/100
140/140 ━━━━━━━━━━━━━━━━━━━━ 154s 1s/step - accuracy: 0.4190 - loss: 1.9296 - 

KeyboardInterrupt: 

In [ ]:
# --- Evaluate ---
loss, mae = model.evaluate(X_val, y_val)
print(f"Validation Loss: {loss:.4f}, MAE: {mae:.4f}")

# --- Predict ---
preds = model.predict(X_val)
print("Sample predictions:", preds[:5])

In [ ]:
# --- Predict on train set ---
train_preds = model.predict(X_train)


# Plot train predictions vs actuals
plt.figure(figsize=(10, 5))
plt.plot(time_train, y_train["target_note"], 'o', label='Actual Notes')
plt.plot(time_train, train_preds[:,1], 'x', label='Predicted Notes')
plt.xlabel('Start Time (or Duration)')
plt.ylabel('Note (MIDI number)')
plt.title('Train Predictions vs Actual Notes')
plt.legend()
plt.grid(True)
plt.show()

Recall that we use a prev 3 day to predict current point. This is characteristics of a lagging controller. This is shown in the plots where the predictions lag behind the actual value. 

In [ ]:
from sequential import *
# Usage:
initial_input = X_val[0]  # Example initial window from validation set
predicted_sequence = sequential_predict(model, initial_input, pred_steps=50)

plt.figure(figsize=(10, 5))

plt.plot(predicted_sequence[:,1], 'o', label='predicted sequence')